In [7]:
import pandas as pd
import time

In [8]:
# load uncorrupted data locally
df_uncorr = pd.read_csv('uncorr.csv')

/Users/xbbu/work/recommendation_system/python_362/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
## Use Pandas
# get top 25 Makes
pandas_st = time.time()
top_makes = list(df_uncorr.Make.value_counts()[:25].index)
pandas_top_makes = time.time()

# get most common color for each Make
pandas_color_dict = {}
for im in top_makes:
    df_im = df_uncorr[['Make', 'Color']].query('Make == "{}"'.format(im))
    top_color = list(df_im.Color.value_counts()[:1].index)
    #print(im, top_color)
    pandas_color_dict[im] = top_color[0]
pandas_top_color = time.time()

# get first ticket issued for each Make
pandas_first_ticket_dict = {}
for im in top_makes:
    df_im = df_uncorr[['Make', 'Issue Date']].query(
        'Make == "{}"'.format(im)).sort_values('Issue Date')
    df_im = df_im.reset_index(drop=True)
    im_first = df_im.at[0, 'Issue Date']
    pandas_first_ticket_dict[im] = im_first
    #print(im, im_first)
pandas_first_ticket = time.time()

In [10]:
## Use SQL
import psycopg2
from psycopg2 import extras

In [11]:
def get_query(query):
    conn = psycopg2.connect(
        "dbname='postgres' user='xbbu' host='localhost' password='' port=5432")
    dict_cur = conn.cursor(cursor_factory=psycopg2.extras.RealDictCursor)
    dict_cur.execute(query)
    
    results = dict_cur.fetchall()
    
    dict_cur.close()
    conn.close()
    
    return results

In [12]:
# query top makes
sql_st = time.time()
top_makes_query = \
    'select count(make), make from car_tickets group by make order by count(make) desc limit 25;'
sql_top_makes = get_query(top_makes_query)
sql_top_makes = [ii['make'] for ii in sql_top_makes]
sql_top_makes_et = time.time()

# get top color for each make
sql_color_dict = {}
for im in sql_top_makes:
    im_query = "select count(color), color from car_tickets where make='{}' group by color order by count(color) desc limit 1;".format(im)
    im_results = get_query(im_query)
    im_results = [ii['color'] for ii in im_results]
    #print(im_results)
    sql_color_dict[im] = im_results[0]
    
sql_top_color_et = time.time()

# get first ticket for each make
sql_first_ticket_dict = {}
for im in sql_top_makes:
    im_query = "select issue_date from car_tickets where make='{}' order by issue_date limit 1;".format(im)
    im_results = get_query(im_query)
    im_results = [ii['issue_date'] for ii in im_results]
    sql_first_ticket_dict[im] = im_results[0]
    
sql_first_ticket_et = time.time()

In [13]:
print('Time elapse for finding top 25 Makes is {:.2f}s and {:.2f}s for Pandas and SQL'.format(
    pandas_top_makes-pandas_st, sql_top_makes_et-sql_st))
print('Time elapse for finding top color for each Make is {:.2f}s and {:.2f} for Pandas and SQL'.format(
    pandas_top_color-pandas_top_makes, sql_top_color_et-sql_top_makes_et))
print('Time elapse for finding first ticket for each Make is {:.2f}s and {:.2f} for Pandas and SQL'.format(
    pandas_first_ticket-pandas_top_color, sql_first_ticket_et-sql_top_color_et))

Time elapse for finding top 25 Makes is 0.41s and 0.65s for Pandas and SQL
Time elapse for finding top color for each Make is 5.68s and 12.34 for Pandas and SQL
Time elapse for finding first ticket for each Make is 8.48s and 40.26 for Pandas and SQL


In [ ]:
"""
For question regarding the expiration plates,
we need to figure out if the ticket is issued in-state or out-state first.
From the existed table, I could not tell.
Here are steps I would take to solve this problem:

Assuming 'RP State Plate' is the state status for the car, while 'Location'
is for where the ticket is issued. We need to get 3rd-party data to determine
if the location is in the 'RP state'.

Once we knew that, we just need convert column 'Plate Expiry Date' to timestamp,
then compare with 'Issue Date' to calculate the fraction of tickets
which have 'Plate Expiry Date < Issue Date'
"""     